In [1]:
from csvcubed.inspect.inspectortable import Inspector

gva = Inspector(
    "~/code/inspector/ess/gross-value-added-per-hour-worked/gross-value-added-per-hour-worked/gross-value-added-per-hour-worked.csv-metadata.json"
)
gmp = Inspector(
    "~/code/inspector/ess/gross-median-weekly-pay/gross-median-weekly-pay/gross-median-weekly-pay.csv-metadata.json"
)

In [2]:
gva.tables

[DataCubeTable(csv_url='gross-value-added-per-hour-worked.csv', title='Gross Value Added per hour worked', shape=<CubeShape.Pivoted: 2>, data_set_uri='gross-value-added-per-hour-worked.csv#dataset'),
 CodeListTable(csv_url='ons-geography-code-list-for-gross-value-added-per-hour-worked.csv', title='ONS Geography Code List for Gross Value Added per hour worked', concept_scheme_uri='ons-geography-code-list-for-gross-value-added-per-hour-worked.csv#code-list'),
 CodeListTable(csv_url='year.csv', title='Year', concept_scheme_uri='year.csv#code-list')]

In [3]:
gmp.tables

[DataCubeTable(csv_url='gross-median-weekly-pay.csv', title='Gross median weekly pay', shape=<CubeShape.Pivoted: 2>, data_set_uri='gross-median-weekly-pay.csv#dataset'),
 CodeListTable(csv_url='ons-geography-code-list-for-gross-median-weekly-pay.csv', title='ONS Geography Code List for Gross median weekly pay', concept_scheme_uri='ons-geography-code-list-for-gross-median-weekly-pay.csv#code-list'),
 CodeListTable(csv_url='year.csv', title='Year', concept_scheme_uri='year.csv#code-list')]

In [4]:
from csvcubed.inspect.inspectorcolumns import SuppressedColumn
gva_columns = gva.tables[0].columns
gva_columns_not_suppressed = {gva_column_title: gva_column_contents for gva_column_title, gva_column_contents in gva_columns.items() if not isinstance(gva_column_contents, SuppressedColumn)}
gva_columns_not_suppressed

{'AREACD': DimensionColumn(dimension=LocalDimension(dimension_uri='gross-value-added-per-hour-worked.csv#dimension/statistical-geography', label='Statistical Geography')),
 'Period': DimensionColumn(dimension=LocalDimension(dimension_uri='gross-value-added-per-hour-worked.csv#dimension/year', label='Year')),
 'Value': PivotedObservationsColumn(unit=LocalUnit(unit_uri='gross-value-added-per-hour-worked.csv#unit/pounds-sterling', label='Pounds Sterling'), measure=LocalMeasure(measure_uri='gross-value-added-per-hour-worked.csv#measure/gross-value-added-per-hour-worked', label='Gross Value Added per hour worked')),
 'Lower Confidence Interval (95%)': AttributeColumn(attribute=LocalAttribute(attribute_uri='gross-value-added-per-hour-worked.csv#attribute/lower-confidence-interval', label='Lower Confidence Interval'), required=False),
 'Upper Confidence Interval (95%)': AttributeColumn(attribute=LocalAttribute(attribute_uri='gross-value-added-per-hour-worked.csv#attribute/upper-confidence-int

In [5]:
gmp_columns = gmp.tables[0].columns
gmp_columns_not_suppressed = {gmp_column_title: gmp_column_contents for gmp_column_title, gmp_column_contents in gmp_columns.items() if not isinstance(gmp_column_contents, SuppressedColumn)}
gmp_columns_not_suppressed

{'AREACD': DimensionColumn(dimension=LocalDimension(dimension_uri='gross-median-weekly-pay.csv#dimension/statistical-geography', label='Statistical Geography')),
 'Period': DimensionColumn(dimension=LocalDimension(dimension_uri='gross-median-weekly-pay.csv#dimension/year', label='Year')),
 'Value': PivotedObservationsColumn(unit=LocalUnit(unit_uri='gross-median-weekly-pay.csv#unit/pounds-sterling', label='Pounds Sterling'), measure=LocalMeasure(measure_uri='gross-median-weekly-pay.csv#measure/gross-median-weekly-pay', label='Gross median weekly pay')),
 'Lower Confidence Interval (95%)': AttributeColumn(attribute=LocalAttribute(attribute_uri='gross-median-weekly-pay.csv#attribute/lower-confidence-interval', label='Lower Confidence Interval'), required=False),
 'Upper Confidence Interval (95%)': AttributeColumn(attribute=LocalAttribute(attribute_uri='gross-median-weekly-pay.csv#attribute/upper-confidence-interval', label='Upper Confidence Interval'), required=False),
 'Observation Statu

In [6]:
gva_df, errors = gva.tables[0].data_cube_repository.get_dataframe(
    gva.tables[0].csv_url, 
    include_suppressed_cols=False, 
    dereference_uris=False)

gva_df.head()


,AREACD,Period,Value,Lower Confidence Interval (95%),Upper Confidence Interval (95%),Observation Status
0,e06000047,2004,22.56,NaN,NaN,NaN
1,e06000047,2020,30.50,NaN,NaN,NaN
2,e06000047,2019,29.97,NaN,NaN,NaN
3,e06000047,2018,29.31,NaN,NaN,NaN
4,e06000047,2017,28.52,NaN,NaN,NaN


In [7]:
gmp_df, errors = gmp.tables[0].data_cube_repository.get_dataframe(
    gmp.tables[0].csv_url, 
    include_suppressed_cols=False, 
    dereference_uris=False)

gmp_df.head()

,AREACD,Period,Value,Lower Confidence Interval (95%),Upper Confidence Interval (95%),Observation Status
0,k02000001,2002,324.8,324.6,325.0,NaN
1,k02000001,2022,532.5,532.3,532.7,NaN
2,k02000001,2021,505.1,504.9,505.3,NaN
3,k02000001,2020,479.1,479.0,479.2,NaN
4,k02000001,2019,479.1,479.0,479.2,NaN


In [12]:
import pandas as pd
from csvcubed.inspect.inspectorcolumns import DimensionColumn

gmp_dimension_col_titles = [
    title 
    for title, column in gmp.tables[0].columns.items() 
    if isinstance(column, DimensionColumn)
]

gmp_gva_merge = pd.merge(
    gmp_df, 
    gva_df, 
    on=gmp_dimension_col_titles, 
    suffixes=(f"_{gmp.tables[0].title}", f"_{gva.tables[0].title}")
).sort_values(gmp_dimension_col_titles)

gmp_gva_merge.head()

,AREACD,Period,Value_Gross median weekly pay,Lower Confidence Interval (95%)_Gross median weekly pay,Upper Confidence Interval (95%)_Gross median weekly pay,Observation Status_Gross median weekly pay,Value_Gross Value Added per hour worked,Lower Confidence Interval (95%)_Gross Value Added per hour worked,Upper Confidence Interval (95%)_Gross Value Added per hour worked,Observation Status_Gross Value Added per hour worked
38,e06000001,2008,370.5,362.3,378.7,NaN,21.47,NaN,NaN,NaN
37,e06000001,2009,363.1,354.2,372.0,NaN,22.38,NaN,NaN,NaN
36,e06000001,2010,383.7,376.1,391.3,NaN,23.47,NaN,NaN,NaN
35,e06000001,2011,368.5,360.3,376.7,NaN,24.68,NaN,NaN,NaN
34,e06000001,2012,391.4,382.5,400.3,NaN,25.43,NaN,NaN,NaN


In [13]:
gva_dimension_col_titles = [
    title 
    for title, column in gva.tables[0].columns.items() 
    if isinstance(column, DimensionColumn)
]

gva_gmp_merge = pd.merge(
    gva_df, 
    gmp_df, 
    on=gva_dimension_col_titles, 
    suffixes=(f"_{gva.tables[0].title}", f"_{gmp.tables[0].title}")
).sort_values(gva_dimension_col_titles)

gva_gmp_merge.head()

,AREACD,Period,Value_Gross Value Added per hour worked,Lower Confidence Interval (95%)_Gross Value Added per hour worked,Upper Confidence Interval (95%)_Gross Value Added per hour worked,Observation Status_Gross Value Added per hour worked,Value_Gross median weekly pay,Lower Confidence Interval (95%)_Gross median weekly pay,Upper Confidence Interval (95%)_Gross median weekly pay,Observation Status_Gross median weekly pay
37,e06000001,2008,21.47,NaN,NaN,NaN,370.5,362.3,378.7,NaN
36,e06000001,2009,22.38,NaN,NaN,NaN,363.1,354.2,372.0,NaN
35,e06000001,2010,23.47,NaN,NaN,NaN,383.7,376.1,391.3,NaN
34,e06000001,2011,24.68,NaN,NaN,NaN,368.5,360.3,376.7,NaN
38,e06000001,2012,25.43,NaN,NaN,NaN,391.4,382.5,400.3,NaN
